In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import requests

In [2]:
cera_url = "https://www.coursera.org"
course_hrefs = {}
lecture_hrefs = {}
week_hrefs = {}
weeks = 0

In [3]:
#Open web browser
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(5)
driver.get(cera_url)

In [4]:
#LOG IN
driver.find_element_by_link_text("LOG IN").click()

In [5]:
#Find the input fields and the submit button
uname = driver.find_element_by_xpath("//input[@name='email']")
passwd = driver.find_element_by_xpath("//input[@name='password']")
submit = driver.find_element_by_xpath("//form//button")

In [6]:
#form = driver.find_element_by_xpath("//form[@name='login']")
#form.get_attribute('action')

In [7]:
#Send the keys to the input fields
uname.send_keys("avisektalks@gmail.com")
passwd.send_keys("Avisek@4326")

In [8]:
submit.click()

In [9]:
driver.find_element_by_xpath("//button[@class='nostyle dropdown']")

<selenium.webdriver.remote.webelement.WebElement (session="b01efd376570d356958d452fc3d652e1", element="0.862186367579961-1")>

In [10]:
driver.find_element_by_xpath("//button[@class='nostyle dropdown']").click()

In [11]:
driver.execute_script("window.scrollTo(0,0)")

In [12]:
soup_lastActive = BeautifulSoup(driver.page_source,'html.parser')

In [13]:
#section_list = soup_lastActive.find_all('section',attrs={"class":"rc-CourseCard with-padding".split()})
modified_section_list = soup_lastActive.find_all('section',attrs={"class":"rc-CourseCard with-padding card-rich-interaction".split()})

In [14]:
for section in modified_section_list:
    print(section.find('h4',attrs={"class":"headline-1-text"}).text)

Machine Learning
Neural Networks for Machine Learning
Introduction to Data Science in Python
Neural Networks and Deep Learning
Neural Networks and Deep Learning
Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization
Structuring Machine Learning Projects
Convolutional Neural Networks
Sequence Models


In [15]:
for section in modified_section_list:
    print(section.find('h4').text)
    check_enrollment = section.find('div',attrs={"class":"rc-CourseEnrollButton"})
    #print(check_enrollment)
    print(None if check_enrollment == None else check_enrollment.find('span').text )

Machine Learning
None
Neural Networks for Machine Learning
None
Introduction to Data Science in Python
None
Neural Networks and Deep Learning
Go to Course
Neural Networks and Deep Learning
None
Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization
Go to Course
Structuring Machine Learning Projects
Enroll
Convolutional Neural Networks
Enroll
Sequence Models
Enroll


In [16]:
for section in modified_section_list:
        check_enrollment = section.find('div',attrs={"class":"rc-CourseEnrollButton"})
        enroll_text = check_enrollment.find('span').text if check_enrollment!=None else 'None'
        print(enroll_text)
        if enroll_text != 'Enroll':
            name = section.find('h4', attrs={"class": "headline-1-text"}).text
            href_link = section.find("a", href=True)['href']
            course_hrefs[name] = href_link           

None
None
None
Go to Course
None
Go to Course
Enroll
Enroll
Enroll


In [17]:
course_hrefs

{'Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization': '/learn/deep-neural-network/home/welcome',
 'Introduction to Data Science in Python': '/learn/python-data-analysis/home/welcome',
 'Machine Learning': '/learn/machine-learning/home/welcome',
 'Neural Networks and Deep Learning': '/learn/neural-networks-deep-learning/home/welcome',
 'Neural Networks for Machine Learning': '/learn/neural-networks/home/welcome'}

In [18]:
course_url = cera_url+course_hrefs['Machine Learning']
course_url

'https://www.coursera.org/learn/machine-learning/home/welcome'

In [19]:
driver.get(course_url)

In [20]:
soup = BeautifulSoup(driver.page_source,'html.parser')
div_weeks = soup.find('div',attrs={'class':'rc-NavigationDrawer'})
weeks = div_weeks.find_all('a',attrs={'class' : "rc-NavigationDrawerLink headline-1-text horizontal-box \
                                                   rc-WeekNavigationItem".split()})

In [21]:
len(weeks)

11

In [22]:
driver.find_element_by_link_text("Week "+str(1)).click()

In [26]:
soup  = BeautifulSoup(driver.page_source,'html.parser')

In [27]:
lesson_divs = soup.find_all('div',{"class":"rc-NamedItemList"})

In [28]:
len(lesson_divs)

8

In [29]:
lessons = [li for li in [div.find_all('li') for div in lesson_divs if div.find('h4').text != 'Review']]

In [35]:
lessons = []
for div in lesson_divs:
    if div.find('h4').text != "Review":
        for li in div.find_all('li'):
            lessons.append(li)
len(lessons)

33

In [40]:
for c,lesson in enumerate(lessons):
    lesson_div = lesson.find('div',attrs={'class':'rc-WeekItemName headline-1-text'.split()})
    #lesson_div.find('span').replace_with('')
    lecture_link = lesson.find('a')['href']
    if "/lecture/" in lecture_link:
        lecture_name = lesson_div.text
    lecture_hrefs[(str(c+1)+". "+lecture_name)] = cera_url + lecture_link

In [41]:
lecture_hrefs

{'1. LectureWelcome to Machine Learning!': 'https://www.coursera.org/learn/machine-learning/lecture/zcAuT/welcome-to-machine-learning',
 '1. Welcome to Machine Learning!': 'https://www.coursera.org/learn/machine-learning/lecture/zcAuT/welcome-to-machine-learning',
 '10. LectureUnsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/1O0Bk/unsupervised-learning',
 '10. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/1O0Bk/unsupervised-learning',
 '11. LectureUnsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/JBzFm/who-are-mentors',
 '11. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/JBzFm/who-are-mentors',
 '12. LectureUnsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/dJCBb/get-to-know-your-classmates',
 '12. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/dJCBb/get-to-know-your-classma

In [43]:
driver.get(lecture_hrefs[list(lecture_hrefs.keys())[0]])

In [44]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [48]:
vid_link = soup.find('video').find('source')['src']

In [49]:
vid_link

'https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4?Expires=1536105600&Signature=MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'

In [56]:
driver.get(vid_link)

In [62]:
_params = {'Expires':'1536105600',
          'Signature':'MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_',
          'Key-Pair-Id':'APKAJLTNE6QMUY6HBC5A'}

In [68]:
vid = requests.get("https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4",params=_params)
print(vid.url)

https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4?Expires=1536105600&Signature=MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A


In [ ]:
vid.contenttype